In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-09 10:30:43.089542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 10:30:43.089613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 10:30:43.091502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 10:30:43.100871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [4]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [5]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True).repeat().unbatch()
is_single_dataset = True
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
it = iter(dataloader)
batch = next(it)
images, image_wrist = [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]

mediapy.show_video(images, fps=10)
# mediapy.show_video(images_wrist, fps=10)

########################################
constructing single val dataset: bridge_dataset
########################################


2024-07-09 10:32:45.815826: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


2024-07-09 10:32:46.146627: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


This browser does not support the video tag.
